# Challenge Sorbonne - DST
# Module VALIDATION

In [3]:
import numpy as np
import pandas as pd
import os
import os
#import rarfile
import networkx as nx
import io
import re
import random
import csv
from datetime import datetime
import tqdm as notebook_tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Subset, Dataset, DataLoader
from torch.optim import AdamW
#!pip install torch_geometric
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
import networkx as nx

from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tqdm as notebook_tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

#env_system = os.environ
#print(env_system)

device is cpu
The current working directory is: c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne\modele_v13


In [4]:
# for Google Colab only
if 'COLAB_RELEASE_TAG' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')

In [5]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

# init du repertoire des données
if current_directory == '/content': # Google Colab
    training_path_dir =   '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_training_set'
    test_path_dir =       '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_test_set'
    train_meta_data =     '/content/drive/MyDrive/Sorbonne_Data_challenge/training_set_metadata.csv'

    model_save_file =     '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/model_sorbonne_weights.pth'
    file_split_training = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/split_val-colab-v12.csv'
    split_char = '/'

    test_init_file =        '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/test_set_to_predict.csv'
    filename_test_results = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/test_prediction.csv'
    filename_trained =      '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/list_hash_trained.csv'

elif current_directory == r"c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v13": #PC1
    split_char = '\\'
    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r"..\Pred_test_v13\list_hash_trained.csv"
    file_split_training = r'..\Pred_test_v13\split_training-pc-v13.csv' # entrainement
    file_split_training = r'..\Pred_test_v13\split_val-pc-v13.csv' # validation

    model_save_file =  r'..\Pred_test_v13\model_sorbonne_weights.pth'
    train_meta_data =  r'..\training_set_metadata.csv'

elif current_directory == r"c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne\modele_v13": #PC2
    split_char = '\\'
    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r"..\Pred_test_v13\list_hash_trained.csv"
    file_split_training = r'..\Pred_test_v13\split_training-pc-v13.csv' # entrainement
    file_split_training = r'..\Pred_test_v13\split_val-pc-v13.csv' # validation

    model_save_file =  r'..\Pred_test_v13\model_sorbonne_weights.pth'
    train_meta_data =  r'..\training_set_metadata.csv'
else:
    print("**** ERROR: init files names and directories not done - root directory and environment not identified")

random_seed = 42
# read CSV input and save in df
df_meta_train = pd.read_csv(train_meta_data, sep=";")
files_meta_list = set(df_meta_train['name'].astype(str))
df_meta_train.head()


The current working directory is: c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne\modele_v13


,name,64-bit execution via heavens gate,64bits,PEB access,accept command line arguments,access the Windows event log,act as TCP client,allocate RW memory,allocate RWX memory,allocate memory,...,winzip,wise,worm,write and execute a file,write clipboard data,write file on Linux,write file on Windows,write pipe,xorcrypt,yoda
0,9fbf213113ba0a18dc2642f83b1201541428fd7951d6a8...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1b35c9dbf3cd9ac60015aaa6cd451c898defa6dac1ff43...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,bf8d307a136a936f7338c1f2eec773c4eb1c802cab77da...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1e51933903f0358c0b635f863368eb15a61cd3442bc5bf...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8a6503fe68d699f8a31531c157e9da931192cd7e3ec809...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_meta_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23102 entries, 0 to 23101
Columns: 454 entries, name to yoda
dtypes: int64(453), object(1)
memory usage: 80.0+ MB


In [7]:
print(filename_trained)
df_files_trained = pd.read_csv(filename_trained, sep=",")
nb_trained_graph = len(df_files_trained)
print("taille :",len(df_files_trained))
df_files_trained.head()


..\Pred_test_v13\list_hash_trained.csv
taille : 115


,files_trained
0,861f2c5f07c9e1c7d24c2e34eb47ff3129cd39a2227a25...
1,86f133e9201e6131cae209f5b3c67f48cd5e23666b9ee0...
2,3c72731f12ebb779bc4c5f0e05a62ca7785d82fe852b2d...
3,414cb126efb9b3e0520ff6504ad2448e383aa284aeca01...
4,73a82d73d442107e3be9f1b10142f9f780746e14e750e3...


In [8]:
# load & check traing split
df_training_split = pd.read_csv(file_split_training, sep=",")
full_file_list = df_training_split[df_training_split['batch'] == 'rep_0']['orign path'].tolist()
print(df_training_split['batch'].unique())
print(full_file_list[:5])
print(df_training_split.info())


['rep_0']
['G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\40936a67037f8fd8e215b045f9cdf9c55840411316a62c85c8b54f75c6b0a5c8.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\368d2b5ccfd49f01942f462037710146ec3ca5ca8a5318a092bc49bfebfe8bad.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\75010f6d21b0b4451b5465ab8f46b385bb1edc15ed50634f4120352edc49cf3a.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\95e944577e686782e9d725e2e79ef9657a3f3b4f3b9cb265487bcd8e55cba95d.json', 'G:\\.shortcut-targets-by-id\\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\\Sorbonne_Data_challenge\\folder_training_set\\72356f26be98c580e23ca7baebe665c781fb2484575b9b05ad38676ad74ffb9e.json']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data

In [9]:
def process_graph_directory(file_directory, val_size=0.1, rep_batch='rep_0', max_file=0):
    """
    Scan the training directory
    get the hash name of the files, check and exclude hash file with error
    retour a list of hash for training, validation and error
    """

    list_train_hash, list_val_hash, list_err_hash  = [], [], []
    file_with_err = 0
    count_files = 0
    #max_file = 0

    #file_split_training = r"C:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\ChallengeSorbonne\split_training-colab.csv"
    df_files_trained = pd.read_csv(filename_trained, sep=",")
    df_training_split = pd.read_csv(file_split_training, sep=",")
    full_file_list = df_training_split[df_training_split['batch']== rep_batch]['orign path'].tolist()

    for full_path_file in full_file_list:
        # test if filename in the trained list
        hash_name = full_path_file.split('.jso')[0]
        hash_name = hash_name.split(split_char)[-1]

        # file already trained
        if hash_name in df_files_trained['files_trained'].values:
        #    print("trained")
            continue

        # files not in metadata
        if hash_name not in df_meta_train['name'].values:
            list_err_hash.append(full_path_file)
            print("meta", hash_name, full_path_file)
            continue

        if os.path.exists(full_path_file):
            file_size = os.path.getsize(full_path_file)
            if file_size/1000 > 100_000: #file too big
                list_err_hash.append(full_path_file)
                print("size", file_size/1000, full_path_file)
                continue
        else:
            list_err_hash.append(full_path_file)
            print("not exist", full_path_file)
            continue

        # check if nb of files is within the max_files allowed
        if max_file != 0 and count_files > max_file:
            list_err_hash.append(full_path_file)
            print("count")
            break

        list_val_hash.append(full_path_file)
        """
        if count_files % (val_size*100) == 0:
            list_val_hash.append(full_path_file)
        else:
            list_train_hash.append(full_path_file)

        count_files += 1
        """
    print(f"Number of files with error: {len(list_err_hash)}")
    return list_train_hash, list_val_hash, list_err_hash

In [10]:
list_train, list_val, list_err = process_graph_directory(training_path_dir, val_size=0.01, rep_batch='rep_0')
print(len(list_val), len(list_train), len(list_err))

size 149418.131 G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\f819c070084c5ab608d6a1d58e7a42bfe8a304aa0003e70a7de72c9fec835f27.json
Number of files with error: 1
81 0 1


# Init modele and functions for validation

In [ ]:
# function to extract and tokenize the graph features
class AssemblyFeatureExtractor:
    def __init__(self):
        self.node_type_vocab = {}
        self.operation_vocab = {}
        self.register_vocab = {}
        self.memory_pattern_vocab = {}
        self.immediate_vocab = {}
        self.UNK_TOKEN = "<UNK>"
        self.max_registers_to_embed = 3  # Embed up to 3 registers

        # Regex patterns for parsing
        self.patterns = {
            'node_type': re.compile(r'([A-Z]+)\s*:'),
            'operation': re.compile(r':\s*([a-z]+)'),
            'registers': re.compile(r'(?:^|\s+)([a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l)(?:$|\s+|,|\])'),
            'memory_ref': re.compile(r'\[(.*?)\]'),
            'immediate': re.compile(r'0x[0-9a-fA-F]+|\d+')
        }

        # Set to store all encountered registers for vocabulary building
        self.all_registers = set()

    def tokenize_line(self, line):
        tokens = {}
        if match := self.patterns['node_type'].search(line):
            tokens['node_type'] = match.group(1)
        if match := self.patterns['operation'].search(line):
            tokens['operation'] = match.group(1)
        if matches := self.patterns['registers'].findall(line):
            tokens['registers'] = [reg.lower() for reg in matches]
            self.all_registers.update(tokens['registers'])
        if match := self.patterns['memory_ref'].search(line):
            tokens['memory_ref'] = match.group(1)
        if match := self.patterns['immediate'].search(line):
            tokens['immediate'] = match.group(0)
        return tokens

    def extract_features(self, digraph):
        all_instruction_features = {}
        for node_id, node_data in digraph.nodes(data=True):
            label = node_data.get('label', '')
            instruction_features_list = []
            instructions = label.split('\n')
            for i, line in enumerate(instructions):
                tokens = self.tokenize_line(line)
                instruction_features = {}

                if 'operation' in tokens:
                    instruction_features['operation'] = tokens['operation']
                if 'node_type' in tokens:
                    instruction_features['node_type'] = tokens['node_type']
                if 'memory_ref' in tokens:
                    instruction_features['memory_ref'] = tokens['memory_ref']
                if 'immediate' in tokens:
                    instruction_features['immediate'] = tokens['immediate']

                # Binary features for register presence (before vocabulary is built)
                for reg in self.all_registers:
                    instruction_features[f'has_register_{reg}'] = 1 if 'registers' in tokens and reg in tokens['registers'] else 0

                # Position in the sequence
                instruction_features['position'] = i
                instruction_features_list.append(instruction_features)

            if len(instruction_features_list) == 1:
                all_instruction_features[node_id] = instruction_features_list[0]
            else:
                all_instruction_features[node_id] = instruction_features_list
        return all_instruction_features

    def build_vocabularies(self, input_data_list):
        node_types = set()
        operations = set()
        registers = set()
        memory_patterns = set()
        immediates = set()

        for item in input_data_list:
            if isinstance(item, nx.DiGraph):
                graph = item
                for _, node_data in graph.nodes(data=True):
                    label = node_data.get('label', '')
                    for line in label.split('\n'):
                        tokens = self.tokenize_line(line)
                        if 'node_type' in tokens:
                            node_types.add(tokens['node_type'])
                        if 'operation' in tokens:
                            operations.add(tokens['operation'])
                        if 'registers' in tokens:
                            registers.update(tokens['registers'])
                        if 'memory_ref' in tokens:
                            memory_patterns.add(tokens['memory_ref'])
                        if 'immediate' in tokens:
                            immediates.add(tokens['immediate'])
            elif isinstance(item, str):
                assembly_code = item
                for line in assembly_code.strip().split('\n'):
                    tokens = self.tokenize_line(line)
                    if 'node_type' in tokens:
                        node_types.add(tokens['node_type'])
                    if 'operation' in tokens:
                        operations.add(tokens['operation'])
                    if 'registers' in tokens:
                        registers.update(tokens['registers'])
                    if 'memory_ref' in tokens:
                        memory_patterns.add(tokens['memory_ref'])
                    if 'immediate' in tokens:
                        immediates.add(tokens['immediate'])

        self.node_type_vocab = {token: i + 1 for i, token in enumerate(sorted(list(node_types)))}
        self.node_type_vocab[self.UNK_TOKEN] = 0
        self.operation_vocab = {token: i + 1 for i, token in enumerate(sorted(list(operations)))}
        self.operation_vocab[self.UNK_TOKEN] = 0
        self.register_vocab = {token: i + 1 for i, token in enumerate(sorted(list(registers)))}
        self.register_vocab[self.UNK_TOKEN] = 0
        self.memory_pattern_vocab = {token: i + 1 for i, token in enumerate(sorted(list(memory_patterns)))}
        self.memory_pattern_vocab[self.UNK_TOKEN] = 0
        self.immediate_vocab = {token: i + 1 for i, token in enumerate(sorted(list(immediates)))}
        self.immediate_vocab[self.UNK_TOKEN] = 0

    def embed_instruction(self, instruction_features, embedding_dim=60):
        """Embeds a single assembly instruction feature dictionary."""
        embedding = []

        # Embed operation (if present)
        op_embedding = np.zeros(embedding_dim)
        if 'operation' in instruction_features and instruction_features['operation'] in self.operation_vocab:
            op_index = self.operation_vocab[instruction_features['operation']]
            op_embedding[op_index % embedding_dim] = 1.0  # Simple one-hot-like hashing
        embedding.extend(op_embedding)

        # Embed node type (if present)
        node_type_embedding = np.zeros(embedding_dim // 2)
        if 'node_type' in instruction_features and instruction_features['node_type'] in self.node_type_vocab:
            node_type_index = self.node_type_vocab[instruction_features['node_type']]
            node_type_embedding[node_type_index % (embedding_dim // 2)] = 1.0
        embedding.extend(node_type_embedding)

        # Embed registers (up to max_registers_to_embed)
        registers_present = [reg.split('_')[-1] for reg in instruction_features if reg.startswith('has_register_') and instruction_features[reg] == 1]
        register_embedding = np.zeros(self.max_registers_to_embed * (embedding_dim // 3)) # Allocate space
        for i, reg in enumerate(registers_present[:self.max_registers_to_embed]):
            if reg in self.register_vocab:
                reg_index = self.register_vocab[reg]
                register_embedding[i * (embedding_dim // 3) + (reg_index % (embedding_dim // 3))] = 1.0
        embedding.extend(register_embedding)

        # Embed memory pattern (if present)
        mem_embedding = np.zeros(embedding_dim // 2)
        if 'memory_ref' in instruction_features and instruction_features['memory_ref'] in self.memory_pattern_vocab:
            mem_index = self.memory_pattern_vocab[instruction_features['memory_ref']]
            mem_embedding[mem_index % (embedding_dim // 2)] = 1.0
        embedding.extend(mem_embedding)

        # Embed immediate (just presence for now)
        has_immediate = 1 if 'immediate' in instruction_features else 0
        embedding.append(has_immediate)

        # Position
        position = instruction_features.get('position', 0)
        embedding.append(position / 10.0) # Normalize position

        return np.array(embedding[:60]) # Truncate to max 60 dimensions

    def embed_graph(self, digraph):
        """Embeds all instructions within the nodes of a digraph."""
        embedded_nodes = {}
        raw_features = self.extract_features(digraph)
        for node_id, features in raw_features.items():
            if isinstance(features, list):  # Multiple instructions in a node
                embedded_nodes[node_id] = [self.embed_instruction(f) for f in features]
            else:  # Single instruction in a node
                embedded_nodes[node_id] = self.embed_instruction(features)
        return embedded_nodes

# Function to create dataloader
#
class AssemblyGraphDataset(Dataset):
    def __init__(self, graph_list, feature_extractor, hash_dim=512):
        self.graph_list = graph_list
        self.feature_extractor = feature_extractor
        self.hash_dim = hash_dim
        self.processed_data = self._process_all_graphs()

    def _process_all_graphs(self):
        processed_data = []
        for graph_item in self.graph_list: # Iterate through the list of graph items
            graph = graph_item # Access the NetworkX graph from the dictionary
            embedded_nodes = self.feature_extractor.embed_graph(graph)

            # Create PyTorch Geometric Data object
            node_features = []
            node_order = sorted(graph.nodes()) # Ensure consistent node order
            for node_id in node_order:
                embedding = embedded_nodes.get(node_id)
                if isinstance(embedding, list):
                    # For simplicity, if a node has multiple instructions, we take the first embedding
                    # You might want a more sophisticated way to handle this (e.g., averaging, using RNN)
                    node_features.append(embedding[0])
                elif embedding is not None:
                    node_features.append(embedding)
                else:
                    node_features.append(np.zeros(60)) # Handle missing embeddings

            # Get edges as a list of tuples
            edges = list(graph.edges())
            if not edges:
                edge_index = torch.empty((2, 0), dtype=torch.long)
            else:
                # Convert node labels to indices if they are not already numeric
                node_to_index = {node: i for i, node in enumerate(graph.nodes())}
                indexed_edges = [(node_to_index[u], node_to_index[v]) for u, v in edges]
                edge_index = torch.tensor(indexed_edges, dtype=torch.long).t().contiguous()

            # Dummy hash encoding (replace with your actual hash encoding)
            hash_encoded = torch.randn(len(node_features), self.hash_dim)

            # Convert lists of numpy arrays to single numpy arrays
            if isinstance(node_features, list):
                node_features_np = np.array(node_features)
            else:
                node_features_np = node_features  # Already a numpy array

            # Convert the numpy arrays to torch tensors
            node_features_tensor = torch.tensor(node_features_np, dtype=torch.float)

            data = Data(x=node_features_tensor,
                        edge_index=edge_index,
                        hash_encoded=hash_encoded)
            processed_data.append(data)

        return processed_data

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]
def collate_fn(batch):
    """Collates a list of PyTorch Geometric Data objects into a Batch object.
    Args:
        batch (list): A list of `torch_geometric.data.Data` objects.
    Returns:
        torch_geometric.data.Batch: A `Batch` object that combines the data
            objects in the `batch`.
    """
    return Batch.from_data_list(batch)

# Define the AssemblyGAT model
class AssemblyGAT(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim=64, output_dim=453, heads=8, dropout=0.6, hash_dim=512):
        super(AssemblyGAT, self).__init__()
        self.conv1 = GATConv(node_feature_dim, hidden_dim, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout
        self.hash_dim = hash_dim

    def forward(self, data):
        """Forward pass through the GAT model"""
        x, edge_index, hash_encoded, batch = data.x, data.edge_index, data.hash_encoded, data.batch

        # First GAT layer with activation and dropout
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Second GAT layer
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

#######################################
# Function to parse digraph string (as in previous example)
def parse_digraph_string(digraph_str):
    """Parse the digraph string to create a NetworkX DiGraph"""
    # Simple parser for the provided format
    G = nx.DiGraph()

    # Regular expressions for node and edge definitions
    node_pattern = re.compile(r'"([^"]+)"\s*\[label\s*=\s*"([^"]+)"\]')
    edge_pattern = re.compile(r'"([^"]+)"\s*->\s*"([^"]+)"')

    # Extract nodes and edges
    for line in digraph_str.strip().split('\n'):
        line = line.strip()

        # Skip the Digraph G { and } lines
        if line == 'Digraph G {' or line == '}':
            continue

        # Parse node definitions
        node_match = node_pattern.search(line)
        if node_match:
            node_id = node_match.group(1)
            label = node_match.group(2)
            G.add_node(node_id, label=label)
            continue

        # Parse edge definitions
        edge_match = edge_pattern.search(line)
        if edge_match:
            source = edge_match.group(1)
            target = edge_match.group(2)
            G.add_edge(source, target)
    return G

def process_batch(file_batch):
    """
    Processes a batch of files.
    Args:
        file_batch (list): A list of file paths.
    """
    graph_list_curr = []
    list_hash = []
    file_with_err = 0

    for full_path_file in file_batch:
        try:
            with open(full_path_file, 'r') as f:
                hash_file = full_path_file.split('.jso')[0]
                hash_file = hash_file.split(split_char)[-1]

                # test if file content has no error tag
                digraph_str = f.read()
                if 'ERROR' in digraph_str:
                    file_with_err += 1
                    continue
                f.close()
                G = parse_digraph_string(digraph_str)
                graph_list_curr.append(G)
                list_hash.append(hash_file)

        except FileNotFoundError:
            print(f"Error: File not found: {full_path_file}")
        except Exception as e:
            print(f"Error processing {full_path_file}: {e}")

    return graph_list_curr, list_hash

def get_metadata_from_hash(hash_list):
    hash_meta_values = []
    for hash_name in hash_list:
        df_y_values = df_meta_train[df_meta_train['name'] == hash_name].copy()
        df_y_values.drop(columns=['name'], inplace=True)
        if len(df_y_values) > 0 :
             hash_meta_values.append(df_y_values.values[0]) # Get the first row's values
        else:
            print(f"{hash_name} not found in metadata")
    return hash_meta_values


def val_model(model, val_loader, y_data, device='cuda'):
    # Move model to device
    model = model.to(device)
    all_pred_label =[]
    all_true_label = []

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
    }
    total = 0
    correct = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            y_data.to(device)

            # Forward pass
            output = model(batch)

            predicted_labels_prob = torch.sigmoid(output)
            prediction_labels = (predicted_labels_prob > 0.5).int()
            all_true_label.append(y_data.cpu())
            all_pred_label.append(prediction_labels.cpu())
            #print("pred & y ", output.shape, y_data.shape )
            total += y_data.shape[0]*y_data.shape[1]
            correct += (prediction_labels == y_data).sum().item()

    accuracy = correct / total if total > 0 else 0.0
    print(f'Accuracy of the network on the validation set: {100*accuracy:.2f}%')
    if not all_true_label:
        print("Warning: No true labels collected during validation. Returning empty tensors.")
        return torch.empty(0, dtype=torch.long).cpu().numpy(), torch.empty(0, dtype=torch.long).cpu().numpy()
    else:
        true_labels_tensor = torch.cat(all_true_label, dim=0) # Concatenate along dimension 0
        pred_labels_tensor = torch.cat(all_pred_label, dim=0) # Concatenate along dimension 0
        return pred_labels_tensor, true_labels_tensor

def run_files_in_batches(full_file_paths, batch_files_size=50, mode='train'):
    """
    Processes files in batches for tokenization and model training/evaluation in a single loop.
    Args:
        full_file_paths (list): List of full file paths to process.
        batch_files_size (int): Number of files to process in each batch for tokenization.
        batch_data_size (int): Batch size for the DataLoader during training/evaluation.
        num_feature_dim (int, optional): Dimension of node features. Defaults to None.
        mode (str): 'train' or other mode (e.g., 'eval'). Defaults to 'train'.
        epochs (int): Number of training epochs per batch (if mode is 'train'). Defaults to 3.
        tokenizer: Tokenizer object with methods like get_tokens_and_counts and fit_from_counts.
        process_batch (callable): Function to process a batch of file paths and return a list of graph-like objects.
        train_model (callable): Function to train the model with a DataLoader and other info.
    """
    print(f"There are {len(full_file_paths)} files for {mode}")

    num_files = len(full_file_paths)
    nb_batch = (num_files // batch_files_size) + 1

    hash_t = []
    all_true_label = []
    all_pred_label = []

    print("**** Tokenization and Dataset Processing & Training")
    for i in range(0, num_files, batch_files_size):
        batch_files = full_file_paths[i:i + batch_files_size]
        batch_num = i // batch_files_size + 1
        print(f"Processing batch {batch_num}/{nb_batch}: {len(batch_files)} files")

        batch_graph_list, hash_t = process_batch(batch_files)
        if not batch_graph_list:
            print("graph list is empty for this batch")
            continue
        
        #    print(hash_t)
        hash_y_list = df_meta_train[df_meta_train['name'].isin(hash_t)].drop(columns=['name']).values
        #print(hash_y_list)
        y_data_tensor = torch.tensor(hash_y_list, dtype=torch.float32)
        
        # Create the AssemblyGraphDataset # Create the DataLoader
        extractor.build_vocabularies(batch_graph_list)
        assembly_dataset = AssemblyGraphDataset(batch_graph_list, extractor)
        dataloader = DataLoader(assembly_dataset, batch_size=batch_data_size, shuffle=False, collate_fn=collate_fn)

        # Train model
        pred_labels, true_labels = val_model(model, dataloader, y_data_tensor, device='cuda' if torch.cuda.is_available() else 'cpu')
        all_true_label.append(true_labels.cpu())
        all_pred_label.append(pred_labels.cpu())

    all_true_label_tensor = torch.cat(all_true_label).cpu().numpy()
    all_pred_label_tensor = torch.cat(all_pred_label).cpu().numpy()
    return all_pred_label_tensor, all_true_label_tensor

In [16]:
###############################################################
# Main init and training sequence
###############################################################
# Batch params
batch_data_size = 2
batch_files_size = 2
node_feature_dim = 60
epochs =  25

# Initialize model
print(f"Run validation on {model_save_file} trained on {nb_trained_graph} graphs\n")
extractor = AssemblyFeatureExtractor()
model = AssemblyGAT(node_feature_dim=node_feature_dim, hidden_dim=64, output_dim=453)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

batch_curr = 'rep_0'
for i in range(0, 400, 500):
  batch_curr = f'rep_{i}'
  print(f"========== batch: {batch_curr} ========== ")
  list_train, list_val, list_err = process_graph_directory(training_path_dir, val_size=5, rep_batch=batch_curr, max_file=500)

  # validate
  all_pred_label_tensor, all_true_label_tensor = run_files_in_batches(list_val, batch_files_size=batch_files_size, mode='validation')
  f1_macro = f1_score(all_true_label_tensor, all_pred_label_tensor, average='macro', zero_division=1)
  print(f"\n**** F1 score: {f1_macro:.5f} \n")



Run validation on ..\Pred_test_v13\model_sorbonne_weights.pth trained on 115 graphs

========== batch: rep_0 ========== 
size 149418.131 G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set\f819c070084c5ab608d6a1d58e7a42bfe8a304aa0003e70a7de72c9fec835f27.json
Number of files with error: 1
There are 81 files for validation
**** Tokenization and Dataset Processing & Training
Processing batch 1/41: 2 files
pred & y  torch.Size([2, 453]) torch.Size([2, 453])
Accuracy of the network on the validation set: 51.88%
Processing batch 2/41: 2 files
pred & y  torch.Size([2, 453]) torch.Size([2, 453])
Accuracy of the network on the validation set: 52.87%
Processing batch 3/41: 2 files
pred & y  torch.Size([2, 453]) torch.Size([2, 453])
Accuracy of the network on the validation set: 51.21%
Processing batch 4/41: 2 files
pred & y  torch.Size([2, 453]) torch.Size([2, 453])
Accuracy of the network on the validation set: 48.45%
Processing batch 5/41: 2